## Novel dataset web scraper

Gathers novel information from novelupdates, http://www.novelupdates.com/,
then cleans the data and arrange everything into a dataset.
The dataset is finally saved as a csv file.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novel_list_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="
novel_page = "http://www.novelupdates.com/?p="

There do not seem to be an easy way to get all novel ids. Therefore, these are gathered from existing list of novels. First the maximum number of novel pages is retrieved and then the novels on these are iterated to get the ids.

In [2]:
# Get the maximum number of pages with novel
def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novel_list_page + '1')
novels_max_pages = get_novel_list_max_pages(page)
print("Pages with novels: " + str(novels_max_pages))

# For testing
novels_max_pages = 2

# Get all novel ids from the novel lists
def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novel_list_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids, columns=['id'])

Pages with novels: 146


In [3]:
def parse_novel_page(id_num):
    #page = requests.get(novel_page + str(id_num))
    page = requests.get(id_num) ## TESTING
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find('div', attrs={'class': 'w-blog-content'})
    if content is None:
        return pd.Series() 
    data = {}
    
    # generatl information
    data['id'] = int(id_num)
    data['name'] = content.div.string.strip()
    data['assoc_names'] = list(content
                               .find('div', attrs={'id': 'editassociated'})
                               .stripped_strings)
    data['authors'] = [author.text.lower()
                for author in content
                  .find('div', attrs={'id': 'showauthors'})
                  .findAll('a')]
    data['org_langauge'] = content.find('div', attrs={'id': 'showlang'}).a.text
    data['genres'] = [genre.text.lower()
                for genre in content
                  .find('div', attrs={'id': 'seriesgenre'})
                  .find_all('a', attrs={'class': 'genre'})]
    data['tags'] = [tag.text.lower()
                for tag in content
                  .find('div', attrs={'id': 'showtags'})
                  .find_all('a')]
    
    # publisher
    data['start_year'] = int(content.find('div', attrs={'id': 'edityear'}).string.strip())
    data['licensed'] = True if content.find('div', attrs={'id': 'showlicensed'}).string.strip() == 'Yes' else False
    data['original_publisher'] = content.find('div', attrs={'id': 'showopublisher'}).a.string.strip().lower()
    data['english_publisher'] = content.find('div', attrs={'id': 'showepublisher'}).a.string.strip().lower()

    # chapters
    volumes_total =
    print(volumes_total)
    #chapters_total =
    #chapters_translated =
    #complete
    #complete_translated =
    
    # translated releases
    #release_freq = 
    #activity_week_rank =
    #activity_month_rank =
    #activity_all_time_rank =
    
    # community
    #on_reading_lists =
    #reading_list_month_rank =
    #reading_list_all_time_rank = 

    rating_text = content.find('span', attrs={'class': 'uvotes'}).text.split(' ')
    data['rating'] = float(rating_text[0][1:])
    data['rating_votes'] = int(rating_text[3])
    
    
    # relations
    #related_series_ids =
    #recommended_series_ids =
    
    time.sleep(1)
    return pd.Series(data)

df = parse_novel_page('http://www.novelupdates.com/series/i-shall-seal-the-heavens/')

#df = pd.merge(df, df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
#df = df.id.apply(lambda x: parse_novel_page(x))
print(df.head)
#df.to_csv('novels.csv', header=True, index=False)

SyntaxError: invalid syntax (<ipython-input-3-796c785348e3>, line 37)